# Part 1

Build a movie review sentiment classifier using GloVe  and RNNs

Tasks:
1. Train a model using GloVE embeddings with Vanilla RNNs
2. Train a model using GloVE embeddings with LSTMs
3. Repeat [1] and [2] with on-the-fly embeddings using torch

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

c:\Users\kshit\.conda\envs\snlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\kshit\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1


## Approach Overview
We will build a movie review sentiment classifier using the IMDB dataset, GloVe embeddings, and RNN-based models. The steps are:
1. **Preprocess the IMDB dataset**: Clean and tokenize the text, split into train/test sets.
2. **Load GloVe embeddings**: Download and prepare GloVe word vectors for use in our models.
3. **Prepare data for PyTorch**: Create datasets and dataloaders for training and evaluation.
4. **Model 1: Vanilla RNN with GloVe**: Build and train a simple RNN using pre-trained GloVe embeddings.
5. **Model 2: LSTM with GloVe**: Build and train an LSTM using pre-trained GloVe embeddings.
6. **Model 3: Vanilla RNN with trainable embeddings**: Use a randomly initialized embedding layer, trained on-the-fly.
7. **Model 4: LSTM with trainable embeddings**: Same as above, but with LSTM.
8. **Evaluate all models**: Compare accuracy and performance on the test set.

## Preprocessing the IMDB Dataset

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
tqdm.pandas()

# Load IMDB dataset
data_path = os.path.join(path, 'IMDB Dataset.csv')
df = pd.read_csv(data_path)
print('Dataset shape:', df.shape)
print(df.head())

# Clean text function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)  # remove HTML tags
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # remove special chars
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['review'] = df['review'].progress_apply(clean_text)

# Encode sentiment
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Train/test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
print('Train:', train_df.shape, 'Test:', test_df.shape)

Dataset shape: (50000, 2)
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


100%|██████████| 50000/50000 [00:02<00:00, 17735.71it/s]

Train: (40000, 3) Test: (10000, 3)


## Loading GloVe Embeddings

In [3]:
import requests
import zipfile

# Download GloVe embeddings (100d)
glove_dir = './glove.6B'
glove_file = os.path.join(glove_dir, 'glove.6B.100d.txt')
if not os.path.exists(glove_file):
    url = 'http://nlp.stanford.edu/data/glove.6B.zip'
    zip_path = 'glove.6B.zip'
    print('Downloading GloVe embeddings...')
    r = requests.get(url, stream=True)
    with open(zip_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    print('Extracting...')
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(glove_dir)
    os.remove(zip_path)
else:
    print('GloVe embeddings already present.')

# Load GloVe vectors into a dictionary
def load_glove_embeddings(glove_path):
    embeddings = {}
    with open(glove_path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

glove_embeddings = load_glove_embeddings(glove_file)
print('Loaded GloVe embeddings:', len(glove_embeddings))

GloVe embeddings already present.
Loaded GloVe embeddings: 400000


## Creating Vocabulary

In [4]:
import torch
from collections import Counter, defaultdict

# Tokenize reviews
def tokenize(text):
    return text.split()

# Build vocabulary manually without torchtext
def build_vocab(texts, max_tokens=20000, specials=['<unk>', '<pad>']):
    # Count all tokens
    counter = Counter()
    for text in texts:
        tokens = tokenize(text)
        counter.update(tokens)
    
    # Get most common tokens
    most_common = counter.most_common(max_tokens - len(specials))
    
    # Build vocab mappings
    token_to_idx = {}
    idx_to_token = {}
    
    # Add special tokens first
    for i, special in enumerate(specials):
        token_to_idx[special] = i
        idx_to_token[i] = special
    
    # Add regular tokens
    for i, (token, count) in enumerate(most_common):
        idx = i + len(specials)
        token_to_idx[token] = idx
        idx_to_token[idx] = token
    
    return token_to_idx, idx_to_token

# Build vocabulary from training data
vocab_size = 20000
specials = ['<unk>', '<pad>']
token_to_idx, idx_to_token = build_vocab(train_df['review'], max_tokens=vocab_size, specials=specials)

print(f'Vocabulary size: {len(token_to_idx)}')
print(f'First 10 tokens: {list(token_to_idx.keys())[:10]}')

# Prepare embedding matrix for GloVe
embedding_dim = 100
embedding_matrix = np.zeros((len(token_to_idx), embedding_dim))

for token, idx in token_to_idx.items():
    vector = glove_embeddings.get(token)
    if vector is not None:
        embedding_matrix[idx] = vector
    else:
        # Random initialization for unknown tokens
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

# Numericalize text function
def numericalize(text, token_to_idx, unk_token='<unk>'):
    tokens = tokenize(text)
    return [token_to_idx.get(token, token_to_idx[unk_token]) for token in tokens]

# Apply numericalization
train_df['input_ids'] = train_df['review'].progress_apply(
    lambda x: numericalize(x, token_to_idx)
)
test_df['input_ids'] = test_df['review'].progress_apply(
    lambda x: numericalize(x, token_to_idx)
)

Vocabulary size: 20000
First 10 tokens: ['<unk>', '<pad>', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it']


100%|██████████| 10000/10000 [00:00<00:00, 27759.84it/s]


## Creating Datasets and Dataloaders

In [5]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.utils.rnn as rnn_utils

class IMDBDataset(Dataset):
    def __init__(self, df, token_to_idx, max_len=200):
        self.input_ids = df['input_ids'].tolist()
        self.labels = df['label'].tolist()
        self.max_len = max_len
        self.pad_idx = token_to_idx['<pad>']
        
    def __len__(self):
        return len(self.input_ids)
        
    def __getitem__(self, idx):
        ids = self.input_ids[idx][:self.max_len]
        length = len(ids)
        if length < self.max_len:
            ids = ids + [self.pad_idx] * (self.max_len - length)
        return torch.tensor(ids, dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.float32)

max_len = 200
batch_size = 128

train_dataset = IMDBDataset(train_df, token_to_idx, max_len=max_len)
test_dataset = IMDBDataset(test_df, token_to_idx, max_len=max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [6]:
import torch.nn as nn

class VanillaRNN(nn.Module):
    def __init__(self, embedding_matrix, token_to_idx, hidden_dim=128, num_layers=1, num_classes=1, dropout=0.2):
        super(VanillaRNN, self).__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        pad_idx = token_to_idx['<pad>']
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=True, padding_idx=pad_idx)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # last hidden state
        out = self.fc(out)
        return self.sigmoid(out).squeeze()

# Instantiate model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
rnn_model = VanillaRNN(embedding_matrix, token_to_idx).to(device)

c:\Users\kshit\.conda\envs\snlp\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


In [7]:
import torch.optim as optim

def train_model(model, train_loader, test_loader, epochs=3, lr=1e-3):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')
    return model

def evaluate_model(model, data_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    print(f'Accuracy: {acc:.4f}')
    return acc

In [8]:
class LSTMModel(nn.Module):
    def __init__(self, embedding_matrix, token_to_idx, hidden_dim=128, num_layers=1, num_classes=1, dropout=0.2):
        super(LSTMModel, self).__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        pad_idx = token_to_idx['<pad>']
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=True, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return self.sigmoid(out).squeeze()

# Instantiate LSTM model
lstm_model = LSTMModel(embedding_matrix, token_to_idx).to(device)

In [9]:
# Vanilla RNN with trainable embeddings
class VanillaRNNTrainable(nn.Module):
    def __init__(self, vocab_size, token_to_idx, embedding_dim=100, hidden_dim=128, num_layers=1, num_classes=1, dropout=0.2):
        super(VanillaRNNTrainable, self).__init__()
        pad_idx = token_to_idx['<pad>']
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return self.sigmoid(out).squeeze()

# LSTM with trainable embeddings
class LSTMTrainable(nn.Module):
    def __init__(self, vocab_size, token_to_idx, embedding_dim=100, hidden_dim=128, num_layers=1, num_classes=1, dropout=0.2):
        super(LSTMTrainable, self).__init__()
        pad_idx = token_to_idx['<pad>']
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return self.sigmoid(out).squeeze()

# Instantiate trainable models
vocab_size = len(token_to_idx)
rnn_trainable_model = VanillaRNNTrainable(vocab_size, token_to_idx).to(device)
lstm_trainable_model = LSTMTrainable(vocab_size, token_to_idx).to(device)

In [12]:
# Train and evaluate Vanilla RNN with GloVe embeddings
print('Training Vanilla RNN (GloVe)...')
rnn_model = train_model(rnn_model, train_loader, test_loader, epochs=10)
print('Evaluating Vanilla RNN (GloVe)...')
rnn_acc = evaluate_model(rnn_model, test_loader)

Training Vanilla RNN (GloVe)...


Epoch 1/10: 100%|██████████| 313/313 [00:02<00:00, 146.30it/s]


Epoch 1, Loss: 0.6848


Epoch 2/10: 100%|██████████| 313/313 [00:02<00:00, 153.20it/s]


Epoch 2, Loss: 0.6862


...

Epoch 8, Loss: 0.6858


Epoch 9/10: 100%|██████████| 313/313 [00:01<00:00, 165.11it/s]


Epoch 9, Loss: 0.6847


Epoch 10/10: 100%|██████████| 313/313 [00:01<00:00, 156.79it/s]



Epoch 10, Loss: 0.6869
Evaluating Vanilla RNN (GloVe)...
Accuracy: 0.5452
Accuracy: 0.5452


In [13]:
# Train and evaluate LSTM with GloVe embeddings
print('Training LSTM (GloVe)...')
lstm_model = train_model(lstm_model, train_loader, test_loader, epochs=10)
print('Evaluating LSTM (GloVe)...')
lstm_acc = evaluate_model(lstm_model, test_loader)

Training LSTM (GloVe)...


Epoch 1/10: 100%|██████████| 313/313 [00:03<00:00, 81.88it/s]


Epoch 1, Loss: 0.6731


Epoch 2/10: 100%|██████████| 313/313 [00:03<00:00, 84.41it/s]


Epoch 2, Loss: 0.6576


...

Epoch 8, Loss: 0.3256


Epoch 9/10: 100%|██████████| 313/313 [00:03<00:00, 81.09it/s]


Epoch 9, Loss: 0.3132


Epoch 10/10: 100%|██████████| 313/313 [00:03<00:00, 80.60it/s]



Epoch 10, Loss: 0.3010
Evaluating LSTM (GloVe)...
Accuracy: 0.8579
Accuracy: 0.8579


In [14]:
# Train and evaluate Vanilla RNN with trainable embeddings
print('Training Vanilla RNN (Trainable Embeddings)...')
rnn_trainable_model = train_model(rnn_trainable_model, train_loader, test_loader, epochs=10)
print('Evaluating Vanilla RNN (Trainable Embeddings)...')
rnn_trainable_acc = evaluate_model(rnn_trainable_model, test_loader)

Training Vanilla RNN (Trainable Embeddings)...


Epoch 1/10: 100%|██████████| 313/313 [00:02<00:00, 149.08it/s]


Epoch 1, Loss: 0.6962


Epoch 2/10: 100%|██████████| 313/313 [00:02<00:00, 151.32it/s]


Epoch 2, Loss: 0.6924


...

Epoch 8, Loss: 0.6371


Epoch 9/10: 100%|██████████| 313/313 [00:02<00:00, 145.00it/s]


Epoch 9, Loss: 0.6170


Epoch 10/10: 100%|██████████| 313/313 [00:02<00:00, 144.30it/s]



Epoch 10, Loss: 0.5881
Evaluating Vanilla RNN (Trainable Embeddings)...
Accuracy: 0.5318
Accuracy: 0.5318


In [15]:
# Train and evaluate LSTM with trainable embeddings
print('Training LSTM (Trainable Embeddings)...')
lstm_trainable_model = train_model(lstm_trainable_model, train_loader, test_loader, epochs=10)
print('Evaluating LSTM (Trainable Embeddings)...')
lstm_trainable_acc = evaluate_model(lstm_trainable_model, test_loader)

Training LSTM (Trainable Embeddings)...


Epoch 1/10: 100%|██████████| 313/313 [00:04<00:00, 75.76it/s]


Epoch 1, Loss: 0.6937


Epoch 2/10: 100%|██████████| 313/313 [00:03<00:00, 78.25it/s]


Epoch 2, Loss: 0.6898


...

Epoch 8, Loss: 0.5900


Epoch 9/10: 100%|██████████| 313/313 [00:03<00:00, 92.58it/s]


Epoch 9, Loss: 0.5080


Epoch 10/10: 100%|██████████| 313/313 [00:03<00:00, 91.98it/s]



Epoch 10, Loss: 0.5698
Evaluating LSTM (Trainable Embeddings)...
Accuracy: 0.5341
Accuracy: 0.5341


In [16]:
# Display all accuracies in a table
import pandas as pd
results = pd.DataFrame({
    'Model': ['Vanilla RNN (GloVe)', 'LSTM (GloVe)', 'Vanilla RNN (Trainable)', 'LSTM (Trainable)'],
    'Accuracy': [rnn_acc, lstm_acc, rnn_trainable_acc, lstm_trainable_acc]
})
display(results)

,Model,Accuracy
0,Vanilla RNN (GloVe),0.5452
1,LSTM (GloVe),0.8579
2,Vanilla RNN (Trainable),0.5318
3,LSTM (Trainable),0.5341
